In [9]:
import pandas as pd
import os
import numpy as np
from datasets import load_dataset, load_metric, set_caching_enabled
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    PrinterCallback,
    Trainer,
    TrainingArguments,
    default_data_collator,
)
from transformers.trainer_utils import set_seed

In [2]:
import wandb

In [3]:
wandb.init()

sweep_config = {
  "name" : "my-sweep",
  "method" : "bayes",
    "metric": {"name": "eval/f1", "goal": "maximize"},
  "parameters" : {
    "learning_rate" :{
      "distribution": "uniform",
      "min": 0.0000001,
      "max": 0.00001
    },
    "num_train_epochs" :{
      "distribution": "int_uniform",
      "min": 1,
      "max": 3
    },
    "per_device_train_batch_size" :{
      "distribution": "int_uniform",
      "min": 8,
      "max": 32
    },
    "weight_decay":{
      "max": 0.02,
      "min": 0.005,
      "distribution": "uniform",
    },
    "warmup_ratio":{ 
      "max": 0.1,
      "min": 0.01,
      "distribution": "uniform"
    }
  }
}
sweep_id = wandb.sweep(sweep_config)

wandb: Currently logged in as: eliottr (use `wandb login --relogin` to force relogin)
2022-04-27 20:04:01.131409: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Create sweep with ID: tjsd3ws1
Sweep URL: https://wandb.ai/eliottr/uncategorized/sweeps/tjsd3ws1


In [4]:
MODEL_NAME = "bert-base-uncased"
METRIC_NAME = "f1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
BATCH_SIZE = 8
MODEL_DIR = "/workspace/models"
DATA_DIR = "/workspace/data"
CACHE_DIR = "/.cache/huggingface"
LOG_DIR = "/workspace/logs"

In [6]:
def tokenize_fn(example):
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, cache_dir=CACHE_DIR)
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=70)  # max length 70 for twitter

def compute_metrics(eval_pred):
    metric = load_metric(METRIC_NAME)
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")


In [7]:
def training_loop():
    with wandb.init() as run:
        # Define model and data

        # DATASET = "movies"
        DATASET = "tweets"
        set_caching_enabled(True)
        set_seed(42)
        NUM_LABELS = 3
        dataset_raw = load_dataset(
                DATA_DIR + "/tweet-sentiment-extraction",
                data_files={"train": "train_for_fine_tune.csv", "test": "test_for_fine_tune.csv"},
            )
        model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS, cache_dir=CACHE_DIR)
        global tokenizer
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR)
        global metric
        metric = load_metric(METRIC_NAME)

        # Prepate the data
        dataset = dataset_raw.map(tokenize_fn, batched=True)
        data_collator = default_data_collator

        train_dataset = dataset["train"]
        eval_dataset = dataset["test"]

        # Train and evaluate the model
        training_args = TrainingArguments(
            f"{MODEL_DIR}/{MODEL_NAME}-finetuned-" + DATASET,
            per_device_train_batch_size=wandb.config.per_device_train_batch_size,
            per_device_eval_batch_size=wandb.config.per_device_train_batch_size,
            learning_rate=wandb.config.learning_rate,
            num_train_epochs=wandb.config.num_train_epochs,
            logging_dir=LOG_DIR,
            warmup_ratio=wandb.config.warmup_ratio,
            weight_decay=wandb.config.weight_decay,
            metric_for_best_model=METRIC_NAME,
            evaluation_strategy="steps",
            logging_steps=50,
            #save_steps=50,
            load_best_model_at_end=True
            # lr_scheduler_type='cosine_with_restarts'
        )
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
            # optimizers=[AdamW(model.parameters(), lr=2e-5), get_cosine_with_hard_restarts_schedule_with_warmup(optimizer=AdamW(model.parameters(), lr=2e-5), num_warmup_steps=300, num_training_steps=3436, num_cycles=3)]
        )

        result_train = trainer.train()
        result_eval = trainer.evaluate()
        trainer.save_model()

In [ ]:
wandb.agent(sweep_id, function=training_loop, count=50)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: pgbqf04v with config:
wandb: 	learning_rate: 6.340994941331681e-06
wandb: 	num_train_epochs: 3
wandb: 	per_device_train_batch_size: 20
wandb: 	warmup_ratio: 0.05049849975627213
wandb: 	weight_decay: 0.016067169519792812
2022-04-27 20:04:12.381465: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Using custom data configuration tweet-sentiment-extraction-26a05391932e4e2c
Reusing dataset csv (/home/eliott.remmer/.cache/huggingface/datasets/csv/tweet-sentiment-extraction-26a05391932e4e2c/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,F1
50,1.108200,1.095607,0.312542
100,1.093300,1.061980,0.433047
150,1.053400,1.008510,0.503555
200,0.982800,0.900186,0.602116
250,0.879400,0.785755,0.670767
300,0.756500,0.681085,0.732422
350,0.707800,0.642355,0.738141
400,0.639500,0.616598,0.751331
450,0.656100,0.595296,0.764201
500,0.614000,0.597090,0.756402


...

Using custom data configuration tweet-sentiment-extraction-26a05391932e4e2c
Reusing dataset csv (/home/eliott.remmer/.cache/huggingface/datasets/csv/tweet-sentiment-extraction-26a05391932e4e2c/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/2 [00:00<?, ?it/s]

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /.cache/huggingface/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.14.1"

Step,Training Loss,Validation Loss,F1
50,1.111200,1.105306,0.268833
100,1.096800,1.090529,0.340385
150,1.095600,1.075498,0.376286
200,1.065400,1.043034,0.411776
250,1.069900,1.019861,0.509984
300,1.015200,0.971733,0.569244
350,0.933900,0.880465,0.637439
400,0.870200,0.822818,0.671032
450,0.816900,0.754193,0.692009
500,0.754500,0.724408,0.685376


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, textID.
***** Running Evaluation *****
  Num examples = 3534
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, textID.
***** Running Evaluation *****
  Num examples = 3534
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, textID.
***** Running Evaluation *****
  Num examples = 3534
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, textID.
***** Running Evaluation *****
  Num examples = 3534
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argu